Basic demonstration of the quantum chemistry code I have written. It seems to be fairly quick, but uses quite a bit of ram. One trick which I certainly did not exploit is that some parts of the environments are simply the harmitian conjugate of other parts, and the environments can therefore be compressed further (and should be twice as cheap to calculate).

In [1]:
using Revise,LinearAlgebra, MPSKit, TensorKit, MPSKitExperimental

LinearAlgebra.BLAS.set_num_threads(1);
TensorKit.TO.disable_cache();
TensorKit.Strided.disable_threads();


FCIDUMP is a well known (though badly documented) format to store the electronic integrals. I have written an experimental parser to load in such files:

In [2]:
(ERI,K,E0,NORB,NELEC,MS2) = parse_fcidump("N2.STO3G.FCIDUMP")

([1.154461655338972 + 0.0im 0.09599004764825508 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.09599004764825508 + 0.0im 0.015609076374304554 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.004885763138763016 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.004885763138763016 + 0.0im;;; 0.09599004764825508 + 0.0im 0.39546474346155136 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.015609076374304554 + 0.0im -0.0010976294332659502 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … -0.00835959931280201 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im -0.00835959931280201 + 0.0im;;; 0.04178887324084891 + 0.0im -0.033247286972367715 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.003247571864194397 + 0.0im 0.0036419884386279613 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … -0.0024350893201443733 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im -0.0024350893201443733 + 0.0im;;; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im

ERI is the Electronic Repulsion Integral, K is the two-body term, E0 is an energy shift, NORB is the number of orbitals (mps length), NELEC is the number of electrons and MS2 is the total spin.

fcidump files can contain orbital symmetries, but I ignore these.

Using these variables, we can instantiate the hamiltonian. Ideally we'd use real numbers, but tensorkit uses a complex default element type for the braidingtensor, and this causes issues.

In [3]:
(ham,_) = fused_quantum_chemistry_hamiltonian(E0,K,ERI,ComplexF64);

half_basis_size = 6


offset 140


onsite 186


3|1 204


2|2 524
1|1|1|1 (1,3) 596
1|1|1|1 (1,3) 644
1|1|1|1 (3,1) 684
1|1|1|1 (3,1) 732


halfbasis 1136


sum([length(o.blocks) for o = opscal_blocks]) = 729
(length((ham[i]).blocks), length((ham[i]).domspaces), length((ham[i]).imspaces)) = 

(10, 1, 12)
(length((ham[i]).blocks), length((ham[i]).domspaces), length((ham[i]).imspaces)) = (45, 12, 20)
(length((ham[i]).blocks), length((ham[i]).domspaces), length((ham[i]).imspaces)) = (44, 20, 32)
(length((ham[i]).blocks), length((ham[i]).domspaces), length((ham[i]).imspaces)) = (56, 32, 50)
(length((ham[i]).blocks), length((ham[i]).domspaces), length((ham[i]).imspaces)) = (78, 50, 72)
(length((ham[i]).blocks), length((ham[i]).domspaces), length((ham[i]).imspaces)) = (173, 72, 58)
(length((ham[i]).blocks), length((ham[i]).domspaces), length((ham[i]).imspaces)) = (95, 58, 43)
(length((ham[i]).blocks), length((ham[i]).domspaces), length((ham[i]).imspaces)) = (50, 43, 16)
(length((ham[i]).blocks), length((ham[i]).domspaces), length((ham[i]).imspaces)) = (21, 16, 7)
(length((ham[i]).blocks), length((ham[i]).domspaces), length((ham[i]).imspaces)) = (7, 7, 1)


The second return value contains some metadata which can be ignored. Instantiate the state:

In [4]:
psp = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((0,0,0)=>1, (1,1//2,1)=>1, (2,0,0)=>1);

left = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((-NELEC,MS2//2,mod1(-NELEC,2))=>1);
right = oneunit(left);

virtual = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((i,s,b)=>1 for i in -NELEC:0, s in 0:1//2:(MS2//2+1), b in (0,1));

st = FiniteMPS(rand,ComplexF64,NORB,psp,virtual;left,right);

We also need to instantiate the environments, because MPSKit will otherwise call the default environment constructor which keeps everything in ram (I think I even disabled that constructor). To create an environment that lives on disk, we can use:

In [5]:
envs = disk_environments(st,ham);

Let's call DMRG!

In [6]:
@time (st,envs) = find_groundstate(st,ham, DMRG2(trscheme=truncdim(100), verbose = true, maxiter=10),envs);

┌ Info: Iteraton 0 error 0.99971363004505
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:112


┌ Info: Iteraton 1 error 0.00158724240224406
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:112


┌ Info: Iteraton 2 error 1.546061869639459e-7
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:112


┌ Info: Iteraton 3 error 5.071867370531891e-10
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:112


┌ Info: Iteraton 4 error 6.146372300008807e-11
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:112


...

┌ Info: Iteraton 6 error 6.146416708929792e-11
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:112


┌ Info: Iteraton 7 error 6.146394504469299e-11
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:112


┌ Info: Iteraton 8 error 6.146394504469299e-11
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:112


┌ Info: Iteraton 9 error 6.146383402239053e-11
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:112


174.896981 seconds (695.93 M allocations: 68.598 GiB, 10.44% gc time, 59.15% compilation time: <1% of which was recompilation)


Have not yet optimized (or checked if it works) the one-site code. I have code for optimizing over the entire unitary basis, but it doesn't work yet with the curent version of the qchem code, so it is not included.